In [1]:
import pandas as pd
import numpy as np
import re

The original dataset

In [5]:
fix_path = r"M:\Python\mocnhien\result\fix_text_updated.csv"
df = pd.read_csv(fix_path)

In [6]:
df['Department'].value_counts()

Department
Thần kinh - Tâm thần    3904
Xét nghiệm              2748
Nội tiết                2149
Truyền nhiễm            1898
Tai mũi họng            1732
Tiêu hóa - Gan mật      1672
Da liễu                 1549
Răng hàm mặt            1493
Hô hấp                  1394
Tiết niệu - Nam khoa    1358
Tim mạch - Mạch máu     1178
Máu                     1038
Ung thư                 1025
Dị ứng                   992
Nhãn khoa                949
Name: count, dtype: int64

In [ ]:
df = df.dropna(subset=['Department', 'Symptom', 'Category'])

In [4]:
df.loc[1].Symptom

'Có 2 hoặc nhiều hơn 2 trạng thái của các nhân cách thay phiên nhau chi phối người bệnh . Các rối loạn về tâm lý này không phải do ảnh hưởng của thuốc hay hóa chất . Các\xa0triệu chứng điển hình mà người mắc bệnh đa nhân cách thường có như sau: . Người mắc bệnh rối loạn đa nhân cách thường có các hành vi, cảm xúc khác bất\xa0bình thường và những những diễn biến tâm lý phức tạp . Khi bị một nhân cách này chi phối, người bệnh có khả năng quên các thông tin quan trọng liên quan đến các nhân cách tiềm ẩn khác, thậm chí không thể xem đó là chứng quên thông thường . Tuy rối loạn đa nhân cách là một căn bệnh rất hiếm gặp, nhưng nếu bạn thấy mình đang có các dấu hiệu của căn bệnh này thì tốt nhất nên đi khám và điều trị sớm \xa0 . Các nhân cách hoàn toàn riêng biệt, tương đối ổn định về nhận thức, quan hệ gia đình, xã hội, khả năng miêu tả thế giới xung quanh '

CleanText

As the sentences distribution are messy in some way. The main here idea is to first, remove all dots, truncate all trailing spaces. Then by the 
logic that all dot must followed by an UpperCased letter, except for the first letter in the sentence 


In [ ]:
# Test function before applying
text = "A.A.A. ."
re.sub(pattern=r"\.", repl=" ", string=text)

In [ ]:
def cleanText(text):
    text = re.sub(pattern=r"\.", repl=" ", string=text) # Remove dots
    text = re.sub(pattern=r"(  +)", repl=" ", string=text) # Remove trailing spaces
    text = re.sub(pattern=r"([A-Z])", repl=r". \1", string=text) # Adding dot before uppercased letters
    
    return text

In [ ]:
df['Symptom'] = df['Symptom'].apply(lambda x:cleanText(x))

In [ ]:
# Remove the leading dot
prefix_dot = df[df['Symptom'].str[0] == '.'].index

for idx in prefix_dot:
    df.loc[idx, 'Symptom'] = df.loc[idx, 'Symptom'].replace(".", "", 1)

Concat duplicate sentences

+ Removing duplicated sentences using set. Consider lowercasing all letters for clearer data :>

In [ ]:
def concatSymptom(text):
   sentences = set(text.lower().split("."))
   return '.'.join([sentence for sentence in sentences if sentence is not None])

In [ ]:
df['Symptom'] = df['Symptom'].apply(lambda x: concatSymptom(x))

Unify Label

In [ ]:
df['Department'].value_counts()

In [ ]:
df['Category'].value_counts()

In [ ]:

def LabelUnifier(label):
    departments = {
    "Thần kinh - Tâm thần": [
        "Thần kinh", "Nội thần kinh", "Ngoại thần kinh", "Tâm thần", "Tâm lý"
    ],
    "Hô hấp": [
        "Hô hấp"
    ],
    "Truyền nhiễm": [
        "Truyền nhiễm"
    ],
    "Tiết niệu - Nam khoa": [
        "Tiết niệu", "Thận tiết niệu", "Nam khoa", "Nam học"
    ],
    "Nội tiết": [
        "Nội tiết", "Nội tiết chuyển hóa"
    ],
    "Tiêu hóa - Gan mật": [
        "Tiêu hóa", "Tiêu hóa - Gan mật"
    ],
    "Tim mạch - Mạch máu": [
        "Tim mạch", "Mạch máu"
    ],
    "Ung thư": [
        "Ung thư", "Ung bướu"
    ],
    "Di truyền": [
        "Di truyền"
    ],
    "Răng hàm mặt": [
        "Răng hàm mặt"
    ],
    "Tai mũi họng": [
        "Tai mũi họng"
    ],
    "Da liễu": [
        "Da liễu", "Da tóc móng"
    ],
    "Nhãn khoa": [
        "Mắt", "Nhãn khoa"
    ],
    "Cơ xương khớp": [
        "Cơ xương khớp"
    ],
    "Dị ứng": [
        "Dị ứng"
    ],
    "Xét nghiệm": [
        "Xét nghiệm"
    ],
    "Sản phụ khoa": [
        "Sản phụ khoa", "Sức khỏe sinh sản", "Hỗ trợ sinh sản ivf", "Mang thai"
    ],
    "Máu": [
        "Máu", "Máu miễn dịch"
    ],
    "Ngoại lồng ngực": [
        "Ngoại lồng ngực"
    ]
}
    for department, keywords in departments.items():
        if label in keywords: return department

    return 0


In [ ]:
df['Department'] = df['Department'].apply(lambda x:LabelUnifier(x))

In [ ]:
df = df[~(df['Department'] == 0)]

In [ ]:
df = df[['Symptom', 'Department', 'Category']]

In [ ]:
df.to_csv(fix_path, index=False)